In [1]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and pl

In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/SemEval-Restaurants/data/test.csv")

In [5]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base")

def tokenize_function(example):
    question_string = f"What is the sentiment of aspect: {example['aspect']}?"

    encoding = tokenizer(
        text=example["text"],
        text_pair=question_string,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/955k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/601k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

In [7]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2521 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    "ufal/robeczech-base",
    num_labels=3
)

model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.793411,0.636111
2,0.946800,0.713614,0.712500
3,0.722500,0.654185,0.744444
4,0.567900,0.647143,0.747222
5,0.567900,0.659445,0.766667
6,0.428400,0.706339,0.763889
7,0.308400,0.788035,0.770833
8,0.236300,0.825900,0.777778
9,0.187400,0.872733,0.770833
10,0.187400,0.884116,0.776389


TrainOutput(global_step=1580, training_loss=0.44475433011598225, metrics={'train_runtime': 1314.0774, 'train_samples_per_second': 19.185, 'train_steps_per_second': 1.202, 'total_flos': 3316544630461440.0, 'train_loss': 0.44475433011598225, 'epoch': 10.0})

In [12]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 0.8259000778198242, 'eval_accuracy': 0.7777777777777778, 'eval_runtime': 9.7573, 'eval_samples_per_second': 73.791, 'eval_steps_per_second': 4.612, 'epoch': 10.0}


In [13]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 0.8113684058189392, 'eval_accuracy': 0.7700831024930748, 'eval_runtime': 4.9844, 'eval_samples_per_second': 72.425, 'eval_steps_per_second': 4.614, 'epoch': 10.0}


In [14]:
trainer.save_model("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA")

In [15]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA")

('/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA/vocab.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA/merges.txt',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA/added_tokens.json',
 '/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA/tokenizer.json')

In [24]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/SemEval-Restaurants/models/robeczech-baseQA")

## Vyhodnocovací metriky

In [17]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [18]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 0.8113684058189392, 'eval_model_preparation_time': 0.005, 'eval_accuracy': 0.7700831024930748, 'eval_runtime': 6.3578, 'eval_samples_per_second': 56.781, 'eval_steps_per_second': 7.235}


In [19]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [23]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 0.8113684058189392, 'eval_model_preparation_time': 0.0051, 'eval_f1': 0.7655320736940592, 'eval_runtime': 5.7796, 'eval_samples_per_second': 62.461, 'eval_steps_per_second': 7.959}


In [21]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [22]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 0.8113684058189392, 'eval_model_preparation_time': 0.0047, 'eval_f1': 0.7006925108874134, 'eval_runtime': 5.8543, 'eval_samples_per_second': 61.664, 'eval_steps_per_second': 7.857}


## Praktické testy

In [25]:
def predict_sentiment(model, tokenizer, aspect, text):
    question_string = f"What is the sentiment of aspect: {aspect}?"

    encoding = tokenizer(
        text=text,
        text_pair=question_string,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [26]:
example_aspect = "power"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "display"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "battery"
example_text = "The power is good, but the display is blurry and dark. The battery was a real surprise, I don't need to charge it during the day!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-1.5475, -1.7954,  3.4872]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 1.8180, -0.8346, -1.1250]])
Predicted sentiment for L2: negative, expected: negative
tensor([[-1.1672, -1.8459,  3.0284]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [27]:
example_aspect = "staff"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "table"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "food"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "beer"
example_text = "When we walked into the restaurant, the staff were very rude and our table hadn't been cleaned. On the other hand, the food was really tasty and the beer was delicious."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 2.2548, -1.7784, -0.7015]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 2.1284, -1.4690, -0.8776]])
Predicted sentiment for R2: negative, expected: negative
tensor([[-1.6377, -1.7474,  3.5702]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-1.6430, -1.7930,  3.6065]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [28]:
example_aspect = "Osaka"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "At EXPO 2025 in Osaka, many companies introduced their latest products. Saremi successfully introduced a new laptop that visitors loved, whereas Babtel's new phone was a disaster, it didn't even work!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[-1.9584,  0.7905,  1.4908]])
Predicted sentiment for M1: positive, expected: neutral
tensor([[-1.0469, -1.7141,  2.9578]])
Predicted sentiment for M2: positive, expected: positive
tensor([[-1.3577, -0.9197,  2.3462]])
Predicted sentiment for M3: positive, expected: negative


### Sada O

In [29]:
example_aspect = "day"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "weather"
example_text = "Today was a really good day, Anna asked me out for a date! Unfortunately the weather was awful, so we had to cancel it."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-1.9164, -1.4838,  3.5122]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 2.3037, -1.5729, -0.9828]])
Predicted sentiment for O2: negative, expected: negative
